In [1]:
import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

Entrez.email = "l.singh@intbio.org"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

45479


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [106]:
# add_histone = (
#     "INSERT INTO histone "
#     "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
#     "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
# )
# add_histone_description = (
#     "INSERT INTO histone_description "
#     "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
#     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
# )
# add_publication = (
#     "INSERT INTO publication "
#     "(id, title, doi, author, year) "
#     "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
# add_alternate_names = (
#     "INSERT INTO alternative_name "
#     "(name, taxonomy, gene, splice, histone) "
#     "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
# )
# add_histone_has_publication = (
#     "INSERT INTO histone_has_publication "
#     "(histone_id, publication_id) "
#     "VALUES (%s, %s)"
# )

# Поменять поле description у H2A.J

Все значения полей у H2A.J попали в summary. Необходимо разнести их на свои места.

In [8]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id = 'H2A.J' "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,expression,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats
0,H2A.J,variant_group,Mammalia,40674,24,H2A,24,H2A.J is a poorly studied replication independ...,null,null,...,null,null,null,null,null,null,null,null,null,null


In [10]:
df["summary"].values[0]

"H2A.J is a poorly studied replication independent H2A histone variant very similar in sequence to canonical H2A histones. It was characterized in human and mouse, similar genes are present in other mammals. It has been implicated in cell senesecence. Genes: In human it is encoded by H2AJ gene on chormomosome 12, in mouse by H2aj gene on chromosome 6. Both genes are intronless. H2aj gene lacks a stem-loop structure at the 3'-UTR but contains a poly (A) signal [nishida_novel_2005]. Similar genes are found in other mammals [contrepois_histone_2017]. Evolution: no studies available. Knock-out: Depletion of H2A.J via RNA interference modifies senesecence-associated chromatin re-structuring and abolishes senesecence-associated secretory phenotype in human fibroblasts subject to ionizing radiation [isermann_histone_2020]. Function, disease: It has been shown that H2A.J accumulates in senescent cells and promotes inflammatory gene expression [isermann_histone_2020,contrepois_histone_2017]. Se

In [12]:
# desc_dict = {
#     "summary": None,
#     "taxonomy": None,
#     "genes": None,
#     "evolution": None,
#     "expression": None,
#     "knock_out": None,
#     "function": None,
#     "sequence": None,
#     "localization": None,
#     "deposition": None,
#     "structure": None,
#     "interactions": None,
#     "disease": None,
#     "caveats": None,
# }

In [20]:
desc_dict = {
    "summary": "H2A.J is a poorly studied replication independent H2A histone variant very similar in sequence to canonical H2A histones. It was characterized in human and mouse, similar genes are present in other mammals. It has been implicated in cell senesecence.",
    "genes": "In human it is encoded by H2AJ gene on chormomosome 12, in mouse by H2aj gene on chromosome 6. Both genes are intronless. H2aj gene lacks a stem-loop structure at the 3'-UTR but contains a poly (A) signal [nishida_novel_2005]. Similar genes are found in other mammals [contrepois_histone_2017].",
    "evolution": "No studies available.",
    "knock_out": "Depletion of H2A.J via RNA interference modifies senesecence-associated chromatin re-structuring and abolishes senesecence-associated secretory phenotype in human fibroblasts subject to ionizing radiation [isermann_histone_2020].",
    "disease": "It has been shown that H2A.J accumulates in senescent cells and promotes inflammatory gene expression [isermann_histone_2020,contrepois_histone_2017].",
    "sequence": "It differs from canonical H2A protein sequences only by an A11V substitution and the presence of an SQK motif near the C-terminus, which is a potential phosphorylation site [talbert_histone_2021].",
    "localization": "In fibroblast irradiation experiments it was shown that H2A.J colocalizes with 53BP1 and is incorporated at the periphery of so-called senesecence-associated chromatin foci (SAHF) [isermann_histone_2020].",
    "deposition": "Unclear.",
    "structure": "Sequence changes lie only within the histone tails, alterations to the C-terminal tail may theoretically alter interactions with linker DNA and H1-histone, but this has not been studied.",
    "interactions": "Colocalizes with 53BP1 [isermann_histone_2020].",
}
desc_dict

{'summary': 'H2A.J is a poorly studied replication independent H2A histone variant very similar in sequence to canonical H2A histones. It was characterized in human and mouse, similar genes are present in other mammals. It has been implicated in cell senesecence.',
 'genes': "In human it is encoded by H2AJ gene on chormomosome 12, in mouse by H2aj gene on chromosome 6. Both genes are intronless. H2aj gene lacks a stem-loop structure at the 3'-UTR but contains a poly (A) signal [nishida_novel_2005]. Similar genes are found in other mammals [contrepois_histone_2017].",
 'evolution': 'No studies available.',
 'knock_out': 'Depletion of H2A.J via RNA interference modifies senesecence-associated chromatin re-structuring and abolishes senesecence-associated secretory phenotype in human fibroblasts subject to ionizing radiation [isermann_histone_2020].',
 'disease': 'It has been shown that H2A.J accumulates in senescent cells and promotes inflammatory gene expression [isermann_histone_2020,co

In [21]:
desk_str = ', '.join([f'{k}="{v}"' for k, v in desc_dict.items()])
query = (
    f"UPDATE histone_description SET {desk_str} "
    "WHERE id = 24"
)
print(query)
cursor.execute(query)

UPDATE histone_description SET summary="H2A.J is a poorly studied replication independent H2A histone variant very similar in sequence to canonical H2A histones. It was characterized in human and mouse, similar genes are present in other mammals. It has been implicated in cell senesecence.", genes="In human it is encoded by H2AJ gene on chormomosome 12, in mouse by H2aj gene on chromosome 6. Both genes are intronless. H2aj gene lacks a stem-loop structure at the 3'-UTR but contains a poly (A) signal [nishida_novel_2005]. Similar genes are found in other mammals [contrepois_histone_2017].", evolution="No studies available.", knock_out="Depletion of H2A.J via RNA interference modifies senesecence-associated chromatin re-structuring and abolishes senesecence-associated secretory phenotype in human fibroblasts subject to ionizing radiation [isermann_histone_2020].", disease="It has been shown that H2A.J accumulates in senescent cells and promotes inflammatory gene expression [isermann_hist

In [22]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id = 'H2A.J' "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,expression,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats
0,H2A.J,variant_group,Mammalia,40674,24,H2A,24,H2A.J is a poorly studied replication independ...,null,In human it is encoded by H2AJ gene on chormom...,...,null,Depletion of H2A.J via RNA interference modifi...,null,It differs from canonical H2A protein sequence...,In fibroblast irradiation experiments it was s...,Unclear.,Sequence changes lie only within the histone t...,Colocalizes with 53BP1 [isermann_histone_2020].,It has been shown that H2A.J accumulates in se...,null


In [23]:
df["summary"].values[0]

'H2A.J is a poorly studied replication independent H2A histone variant very similar in sequence to canonical H2A histones. It was characterized in human and mouse, similar genes are present in other mammals. It has been implicated in cell senesecence.'

In [24]:
# Make sure data is committed to the database
conn.commit()

# Close connections

In [25]:
cursor.close()
conn.close()
tunnel.stop()